In [1]:
import os
import re
import sys
import copy

ROOT = os.path.join(os.path.dirname(os.getcwd()), 'backend')
if ROOT not in sys.path:
    sys.path.append(ROOT)
    
from moz_sql_parser import parse as ps
sys.setrecursionlimit(1000000)
import json
import os
import pandas as pd
from termcolor import colored
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import AgglomerativeClustering
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [2]:
from app.dataService.dataService import DataService

In [3]:
# Generate SQL and the execution results from the plain text
text = "films and film prices that cost below 10 dollars"
db_id = "cinema"
dataService = DataService("spider")

=== begin loading model ===
=== finish loading model ===


In [4]:
# read spider dataset
dataset_type = "train_spider"
# dataset_type="dev"
with open(f"../backend/app/data/dataset/spider/{dataset_type}.json", "r") as f:
    spider_data = json.load(f)
df = pd.DataFrame(spider_data)

## Parse SQL using moz-parser

In [5]:
df.head()

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,department_management,SELECT count(*) FROM head WHERE age > 56,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...",How many heads of the departments are older th...,"[How, many, heads, of, the, departments, are, ...","{'from': {'table_units': [['table_unit', 1]], ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","List the name, born state and age of the heads...","[List, the, name, ,, born, state, and, age, of...","{'from': {'table_units': [['table_unit', 1]], ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","List the creation year, name and budget of eac...","[List, the, creation, year, ,, name, and, budg...","{'from': {'table_units': [['table_unit', 0]], ..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...","[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...",What are the maximum and minimum budget of the...,"[What, are, the, maximum, and, minimum, budget...","{'from': {'table_units': [['table_unit', 0]], ..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...",What is the average number of employees of the...,"[What, is, the, average, number, of, employees...","{'from': {'table_units': [['table_unit', 0]], ..."


In [6]:
from app.dataService.utils.processSQL import decode_sql
import re

cond_list = []
for rowid, row in df.iterrows():
    if rowid != 3153:
        db_id = row["db_id"]
        sql = row["query"]
        parsed = dataService.parsesql(sql, db_id)
        table = parsed["table"]
        sql_parse = parsed["sql_parse"]

#         print(decode_sql.decode_from(sql_parse["from"], table))
        print(decode_sql.decode_union(sql_parse["union"], table))
#         if sql_parse["intersect"] is not None:
#             print(sql_parse["intersect"])
#             print()
#         if len(parsed["sql_parse"]["having"])>1:
#             print(parsed["sql_parse"]["having"])
#         decode_sql.decode_having(sql_parse["having"], table)
#         if len(sql_parse['having']) >= 3:
#             print(colored(sql_parse['having'], "red"))
#             print()
#         if len(sql_parse['where'])>=3:
#             print(colored(sql_parse['where'], "yellow"))
#             print()
#         if len(sql_parse['from']["conds"])>3:
#             print(colored(sql_parse['from']["conds"], "blue"))
#             print()

#         cond_list.append(len(sql_parse['having']))
#         cond_list.append(len(sql_parse['where']))
#         cond_list.append(len(sql_parse['from']["conds"]))
        
#         if len(sql_parse['having']) >=2:
#             print(sql_parse['having'])
#         elif len(sql_parse["having"]) == 1:
#             print(sql_parse['having'])
        # print(decode_sql.decode_groupby(sql_parse["groupBy"], table))
        # print(decode_sql.decode_orderby(sql_parse["orderBy"], table))
        # print(colored(decode_sql.decode_limit(sql_parse["limit"], table), "red"))
        # decode_sql.decode_select(parsed["sql_parse"]["select"], table)

# print(f"conditions: min: {np.min(cond_list)}, max: {np.max(cond_list)}")

{'table_units': [('table_unit', 'head')], 'conds': []}
{'table_units': [('table_unit', 'head')], 'conds': []}
{'table_units': [('table_unit', 'department')], 'conds': []}
{'table_units': [('table_unit', 'department')], 'conds': []}
{'table_units': [('table_unit', 'department')], 'conds': []}
{'table_units': [('table_unit', 'head')], 'conds': []}
{'table_units': [('table_unit', 'department'), ('table_unit', 'management'), ('table_unit', 'head')], 'conds': [(False, '=', ('none', ('none', 'department: department id', ''), None), None, None), 'and', (False, '=', ('none', ('none', 'management: head id', ''), None), None, None)]}
{'table_units': [('table_unit', 'head')], 'conds': []}
{'table_units': [('table_unit', 'department')], 'conds': []}
{'table_units': [('table_unit', 'department'), ('table_unit', 'management')], 'conds': [(False, '=', ('none', ('none', 'department: department id', ''), None), None, None)]}
{'table_units': [('table_unit', 'management')], 'conds': []}
{'table_units': [

{'table_units': [('table_unit', 'enzyme'), ('table_unit', 'medicine enzyme interaction')], 'conds': [(False, '=', ('none', ('none', 'enzyme: id', ''), None), None, None)]}
{'table_units': [('table_unit', 'enzyme'), ('table_unit', 'medicine enzyme interaction')], 'conds': [(False, '=', ('none', ('none', 'enzyme: id', ''), None), None, None)]}
{'table_units': [('table_unit', 'medicine enzyme interaction'), ('table_unit', 'medicine'), ('table_unit', 'enzyme')], 'conds': [(False, '=', ('none', ('none', 'medicine enzyme interaction: medicine id', ''), None), None, None), 'and', (False, '=', ('none', ('none', 'medicine enzyme interaction: enzyme id', ''), None), None, None)]}
{'table_units': [('table_unit', 'medicine enzyme interaction'), ('table_unit', 'medicine'), ('table_unit', 'enzyme')], 'conds': [(False, '=', ('none', ('none', 'medicine enzyme interaction: medicine id', ''), None), None, None), 'and', (False, '=', ('none', ('none', 'medicine enzyme interaction: enzyme id', ''), None), 

{'table_units': [('table_unit', 'schedule'), ('table_unit', 'film')], 'conds': [(False, '=', ('none', ('none', 'schedule: film id', ''), None), None, None)]}
{'table_units': [('table_unit', 'schedule'), ('table_unit', 'film'), ('table_unit', 'cinema')], 'conds': [(False, '=', ('none', ('none', 'schedule: film id', ''), None), None, None), 'and', (False, '=', ('none', ('none', 'schedule: cinema id', ''), None), None, None)]}
{'table_units': [('table_unit', 'film')], 'conds': []}
{'table_units': [('table_unit', 'schedule'), ('table_unit', 'film')], 'conds': [(False, '=', ('none', ('none', 'schedule: film id', ''), None), None, None)]}
{'table_units': [('table_unit', 'cinema')], 'conds': []}
{'table_units': [('table_unit', 'cinema')], 'conds': []}
{'table_units': [('table_unit', 'film')], 'conds': []}
{'table_units': [('table_unit', 'film')], 'conds': []}
{'table_units': [('table_unit', 'customers'), ('table_unit', 'discount coupons')], 'conds': [(False, '=', ('none', ('none', 'customers:

{'table_units': [('table_unit', 'rating'), ('table_unit', 'movie'), ('table_unit', 'reviewer')], 'conds': [(False, '=', ('none', ('none', 'rating: movie id', ''), None), None, None), 'and', (False, '=', ('none', ('none', 'rating: reviewer id', ''), None), None, None)]}
{'table_units': [('table_unit', 'rating'), ('table_unit', 'movie'), ('table_unit', 'reviewer')], 'conds': [(False, '=', ('none', ('none', 'rating: movie id', ''), None), None, None), 'and', (False, '=', ('none', ('none', 'rating: reviewer id', ''), None), None, None)]}
{'table_units': [('table_unit', 'rating'), ('table_unit', 'movie'), ('table_unit', 'reviewer')], 'conds': [(False, '=', ('none', ('none', 'rating: movie id', ''), None), None, None), 'and', (False, '=', ('none', ('none', 'rating: reviewer id', ''), None), None, None)]}
{'table_units': [('table_unit', 'rating'), ('table_unit', 'movie'), ('table_unit', 'reviewer')], 'conds': [(False, '=', ('none', ('none', 'rating: movie id', ''), None), None, None), 'and', 

{'table_units': [('table_unit', 'employees')], 'conds': []}
{'table_units': [('table_unit', 'employees')], 'conds': []}
{'table_units': [('table_unit', 'employees')], 'conds': []}
{'table_units': [('table_unit', 'locations')], 'conds': []}
{'table_units': [('table_unit', 'locations')], 'conds': []}
{'table_units': [('table_unit', 'job history')], 'conds': []}
{'table_units': [('table_unit', 'job history')], 'conds': []}
{'table_units': [('table_unit', 'job history')], 'conds': []}
{'table_units': [('table_unit', 'job history')], 'conds': []}
{'table_units': [('table_unit', 'employees'), ('table_unit', 'departments'), ('table_unit', 'locations'), ('table_unit', 'countries')], 'conds': [(False, '=', ('none', ('none', 'employees: department id', ''), None), None, None), 'and', (False, '=', ('none', ('none', 'departments: location id', ''), None), None, None), 'and', (False, '=', ('none', ('none', 'locations: country id', ''), None), None, None)]}
{'table_units': [('table_unit', 'employees

{'table_units': [('table_unit', 'nurse')], 'conds': []}
{'table_units': [('table_unit', 'nurse')], 'conds': []}
{'table_units': [('table_unit', 'undergoes'), ('table_unit', 'nurse')], 'conds': [(False, '=', ('none', ('none', 'undergoes: assisting nurse', ''), None), None, None)]}
{'table_units': [('table_unit', 'undergoes'), ('table_unit', 'nurse')], 'conds': [(False, '=', ('none', ('none', 'undergoes: assisting nurse', ''), None), None, None)]}
{'table_units': [('table_unit', 'medication')], 'conds': []}
{'table_units': [('table_unit', 'medication')], 'conds': []}
{'table_units': [('table_unit', 'physician'), ('table_unit', 'prescribes')], 'conds': [(False, '=', ('none', ('none', 'physician: employee id', ''), None), None, None)]}
{'table_units': [('table_unit', 'physician'), ('table_unit', 'prescribes')], 'conds': [(False, '=', ('none', ('none', 'physician: employee id', ''), None), None, None)]}
{'table_units': [('table_unit', 'affiliated with')], 'conds': []}
{'table_units': [('tab

{'table_units': [('table_unit', 'player')], 'conds': []}
{'table_units': [('table_unit', 'college')], 'conds': []}
{'table_units': [('table_unit', 'college')], 'conds': []}
{'table_units': [('table_unit', 'player')], 'conds': []}
{'table_units': [('table_unit', 'player')], 'conds': []}
{'table_units': [('table_unit', 'player')], 'conds': []}
{'table_units': [('table_unit', 'player')], 'conds': []}
{'table_units': [('table_unit', 'tryout')], 'conds': []}
{'table_units': [('table_unit', 'tryout')], 'conds': []}
{'table_units': [('table_unit', 'tryout')], 'conds': []}
{'table_units': [('table_unit', 'tryout')], 'conds': []}
{'table_units': [('table_unit', 'tryout')], 'conds': []}
{'table_units': [('table_unit', 'tryout')], 'conds': []}
{'table_units': [('table_unit', 'tryout')], 'conds': []}
{'table_units': [('table_unit', 'tryout')], 'conds': []}
{'table_units': [('table_unit', 'player')], 'conds': []}
{'table_units': [('table_unit', 'player')], 'conds': []}
{'table_units': [('table_unit

{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'student')], 'conds': []}
{'table_units': [('table_unit', 'voting record')], 'conds': []}
{'table_units': [('table_unit', 'voting record')], 'conds': []}
{'table_units': [('table_unit', 'voting record')], 'conds': []}
{'table_units': [('table_unit', 'voting record')], 'co

{'table_units': [('table_unit', 'customers'), ('table_unit', 'customer orders')], 'conds': [(False, '=', ('none', ('none', 'customers: customer id', ''), None), None, None)]}
{'table_units': [('table_unit', 'order items'), ('table_unit', 'products')], 'conds': [(False, '=', ('none', ('none', 'order items: product id', ''), None), None, None)]}
{'table_units': [('table_unit', 'order items'), ('table_unit', 'products')], 'conds': [(False, '=', ('none', ('none', 'order items: product id', ''), None), None, None)]}
{'table_units': [('table_unit', 'order items'), ('table_unit', 'products')], 'conds': [(False, '=', ('none', ('none', 'order items: product id', ''), None), None, None)]}
{'table_units': [('table_unit', 'order items'), ('table_unit', 'products')], 'conds': [(False, '=', ('none', ('none', 'order items: product id', ''), None), None, None)]}
{'table_units': [('table_unit', 'addresses')], 'conds': []}
{'table_units': [('table_unit', 'addresses')], 'conds': []}
{'table_units': [('ta

{'table_units': [('table_unit', 'architect'), ('table_unit', 'mill')], 'conds': [(False, '=', ('none', ('none', 'architect: id', ''), None), None, None)]}
{'table_units': [('table_unit', 'mill')], 'conds': []}
{'table_units': [('table_unit', 'mill'), ('table_unit', 'architect')], 'conds': [(False, '=', ('none', ('none', 'mill: architect id', ''), None), None, None)]}
{'table_units': [('table_unit', 'architect'), ('table_unit', 'bridge')], 'conds': [(False, '=', ('none', ('none', 'architect: id', ''), None), None, None)]}
{'table_units': [('table_unit', 'architect'), ('table_unit', 'mill')], 'conds': [(False, '=', ('none', ('none', 'architect: id', ''), None), None, None)]}
{'table_units': [('table_unit', 'architect'), ('table_unit', 'bridge')], 'conds': [(False, '=', ('none', ('none', 'architect: id', ''), None), None, None)]}
{'table_units': [('table_unit', 'bridge')], 'conds': []}
{'table_units': [('table_unit', 'mill')], 'conds': []}
{'table_units': [('table_unit', 'mill'), ('table_

In [7]:
isinstance(2.4, float)

True